# recomendation system (user - based)

In [2]:
# user based means : 
#Example: If Alice and Bob both liked "Movie A" and "Movie B", and Bob also liked "Movie C", then recommend "Movie C" to Alice.

In [3]:
import numpy as np
import pandas as pd

In [4]:
# load dataset 
df = pd.read_csv('Ratings.csv')
print(df.head())
print(df.shape)


   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6
(1149780, 3)


# cleaning the dataset and processiong

In [6]:
# checking null objects 
print(df.isnull().sum())

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


In [7]:
df.groupby('Book-Rating').size()
# 0 rated means user doesnt give any reviews he just viewed or borrowed 
# so we need to remove records which as book ratings has 0
df_clean = df[df['Book-Rating']>0]
df_clean = df_clean.reset_index(drop = True)

In [8]:
df.groupby('Book-Rating').size()

Book-Rating
0     716109
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
dtype: int64

In [9]:
# processed data 
df_clean.groupby('Book-Rating').size()

Book-Rating
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
dtype: int64

# now creating a pivot table 

In [11]:
# were ecah book has an seperate columns 
df_sample = df_clean.sample(10000)

user_matrix = df_sample.pivot_table(index = 'User-ID',columns = 'ISBN',values = 'Book-Rating')
print(user_matrix.shape)

(6338, 8725)


In [26]:
user_matrix = user_matrix.fillna(0)
user_matrix


ISBN,0 09 944678 2,0 671 74251 5,000000000,0000000000,0000121214523,000105337X,0002224216,0002226324,0002245841,0002255081,...,ISBN000617024,ISBN073941625,ISBN084077820,ISBN887255150,N0553249533>>,NAMORZAID,NONEAVAILABLE,O312975651,O895771594,O97I88OIO7
User-ID,,,,,,,,,,,,,,,,,,,,,
242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Calculate cosine similarity between users

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
user_similarity = cosine_similarity(user_matrix)
# convert to a dataframe 
user_similarity_df = pd.DataFrame(user_similarity,index = user_matrix.index,columns =user_matrix.index)

In [58]:
def get_top_n_similar_users(user_id, similarity_df, n=5):
    similar_users = similarity_df[user_id].drop(user_id)
    similar_users = similar_users[similar_users > 0]  # drop users with zero similarity
    top_users = similar_users.sort_values(ascending=False).head(n)
    return top_users



In [62]:
target_user_id = user_matrix.index[0]  # or any other User-ID

top_similar_users = get_top_n_similar_users(target_user_id, user_similarity_df, n=5)
print("Top similar users:\n", top_similar_users)


Top similar users:
 Series([], Name: 242, dtype: float64)


In [64]:
# Books the target user has rated
target_user_ratings = user_matrix.loc[target_user_id]
books_rated_by_target = target_user_ratings[target_user_ratings > 0].index


In [72]:
# Ratings from similar users
similar_users_ratings = user_matrix.loc[top_similar_users.index]

# Keep only books not rated by the target user
unrated_books = user_matrix.columns.difference(books_rated_by_target)
similar_users_unrated = similar_users_unrated.fillna(0)



In [74]:
# Weighted rating: each rating * similarity score
weighted_ratings = similar_users_unrated.T.dot(top_similar_users)

# Normalize by similarity scores sum
similarity_sum = top_similar_users.sum()
recommendation_scores = weighted_ratings / similarity_sum


In [76]:
top_recommendations = recommendation_scores.sort_values(ascending=False).head(10)
print("Recommended books:\n", top_recommendations)


Recommended books:
 ISBN
0 09 944678 2   NaN
0 671 74251 5   NaN
000000000       NaN
0000000000      NaN
0000121214523   NaN
000105337X      NaN
0002224216      NaN
0002226324      NaN
0002245841      NaN
0002255081      NaN
dtype: float64
